In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
importlib.reload(kgs)
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
from IPython.display import HTML, display, clear_output
kgs.set_float32(True)
pack_cuda._ensure_initialized()


local
local
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENABLE_SEPARATION -cubin /mnt/d//packing/temp/pack_cuda_saved.cu -o /mnt/d//packing/temp/pack_cuda_crystal.cubin
ptxas info    : 0 bytes gmem, 1172 bytes cmem[3]
ptxas info    : Compiling entry function 'multi_boundary_distance_list_total' for 'sm_89'
ptxas info    : Function properties for multi_boundary_distance_list_total
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 36 registers, 404 bytes cmem[0], 16 bytes cmem[2]
ptxas info    : Compiling entry function 'multi_boundary_list_total' for 'sm_89'
ptxas info    : Function properties for multi_boundary_list_tot

In [2]:
%load_ext pyinstrument

In [3]:
#%%pyinstrument
import pack_ga
kgs.enable_module_profiling(pack_ga)
import pack_runner
runner = pack_runner.baseline_runner(fast_mode=False)
runner.use_missing_value = True
runner.base_ga.n_generations = 5
runner.base_ga.do_legalize = False
runner.base_ga.N_trees_to_do = np.array([40])
runner.run()


stop final relax at some point
Processing class: Crossover
  Added method: Crossover.__init__
  Added method: Crossover._apply_transformation
  Added method: Crossover._check_constraints
  Added method: Crossover._check_types
  Added method: Crossover._do_move_vec
  Added method: Crossover.check_constraints
  Added method: Crossover.do_move
  Added method: Crossover.do_move_vec
Processing class: GA
  Added method: GA.__init__
  Added method: GA._check_constraints
  Added method: GA._check_types
  Added method: GA._relax_and_score
  Added method: GA._score
  Added method: GA.check_constraints
  Added method: GA.run
Processing class: Initializer
  Added method: Initializer.__init__
  Added method: Initializer._check_constraints
  Added method: Initializer._check_types
  Added method: Initializer.check_constraints
  Added method: Initializer.initialize_population
Processing class: InitializerRandomJiggled
  Added method: InitializerRandomJiggled.__init__
  Added method: InitializerRandomJ

/mnt/d/packing/code/analysis/../core/pack_dynamics.py:61: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  x0 = from_dlpack(x0.toDlpack())
/mnt/d/packing/code/analysis/../core/pack_dynamics.py:93: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  return from_dlpack(tmp_cost[:N].toDlpack()), from_dlpack(res.toDlpack())


Generation 0, Trees 40, Best cost: 0.16849418, Est: 0.42123547, h: 3.700000
Generation 1, Trees 40, Best cost: 0.09710957, Est: 0.24277394, h: 3.700000
Generation 2, Trees 40, Best cost: 0.08559522, Est: 0.21398804, h: 3.700000
Generation 3, Trees 40, Best cost: 0.07632724, Est: 0.19081811, h: 3.700000
Generation 4, Trees 40, Best cost: 0.05988770, Est: 0.14971925, h: 3.700000
Runtime: 26.9s


In [4]:
kgs.print_module_profile()


Timer unit: 1e-09 s

Total time: 0.0532879 s
File: /mnt/d/packing/code/analysis/../core/kaggle_support.py
Function: BaseClass.check_constraints at line 105

Line #      Hits         Time  Per Hit   % Time  Line Contents
   105                                               def check_constraints(self, debugging_mode_offset = 0):
   106                                                   global debugging_mode
   107        40      27928.0    698.2      0.1          debugging_mode = debugging_mode+debugging_mode_offset
   108        40      12198.0    304.9      0.0          try:
   109        40      11809.0    295.2      0.0              if debugging_mode > 0:
   110        40    3148301.0  78707.5      5.9                  self._check_types()
   111        40   50050964.0 1.25e+06     93.9                  self._check_constraints()
   112        40      17554.0    438.9      0.0              return
   113                                                   finally:
   114        40      191